# Day13_0 별첨: SVR (Support Vector Regression) - 정답

---

In [ ]:
# 환경 설정
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.datasets import fetch_california_housing

import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

# 데이터 준비
housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['MedHouseVal'] = housing.target
df_sample = df.sample(n=3000, random_state=42)

X = df_sample.drop('MedHouseVal', axis=1)
y = df_sample['MedHouseVal']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("데이터 준비 완료!")

---

## Q1. 기본 SVR 학습 ⭐

**문제**: RBF 커널의 SVR 모델을 학습하고 R2 스코어를 계산하세요.

In [ ]:
# 정답 코드
# 1. SVR 모델 생성 (기본 RBF 커널)
svr_model = SVR(kernel='rbf')

# 2. 학습 (스케일링된 데이터 사용)
svr_model.fit(X_train_scaled, y_train)

# 3. 예측
y_pred = svr_model.predict(X_test_scaled)

# 4. R2 스코어 계산
r2 = r2_score(y_test, y_pred)
print(f"R2 Score: {r2:.4f}")

In [ ]:
# 검증
assert r2 > 0.5, "R2가 0.5보다 작습니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. SVR(kernel='rbf')로 모델 생성
2. 스케일링된 데이터로 학습 (SVR 필수)
3. r2_score로 평가

**핵심 개념**:
- SVR은 스케일링 필수
- RBF 커널이 기본이며 비선형 패턴 학습

**흔한 실수**:
- 스케일링 없이 학습 (성능 저하)

**실무 팁**:
- 항상 StandardScaler 적용 후 SVR 학습

---

## Q2. epsilon 설정 ⭐

**문제**: epsilon=0.5인 SVR 모델을 학습하고 RMSE를 계산하세요.

In [ ]:
# 정답 코드
# 1. SVR 모델 생성 (epsilon=0.5)
svr_eps = SVR(kernel='rbf', epsilon=0.5)

# 2. 학습
svr_eps.fit(X_train_scaled, y_train)

# 3. 예측
y_pred_eps = svr_eps.predict(X_test_scaled)

# 4. RMSE 계산
rmse = np.sqrt(mean_squared_error(y_test, y_pred_eps))
print(f"RMSE: {rmse:.4f}")

In [ ]:
# 검증
assert rmse < 1.0, "RMSE가 1.0보다 큽니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. epsilon=0.5로 오차 허용 범위 설정
2. np.sqrt(MSE)로 RMSE 계산

**핵심 개념**:
- epsilon: ε-tube의 폭 (오차 허용 범위)
- 큰 epsilon: 더 많은 오차 허용, 일반화 경향

**실무 팁**:
- 타겟 스케일에 맞게 epsilon 설정 (보통 표준편차의 10-20%)

---

## Q3. C 파라미터 비교 ⭐⭐

**문제**: C=0.1, 1, 10, 100 네 가지 값으로 SVR을 학습하고 R2 스코어를 비교하세요.

In [ ]:
# 정답 코드
C_values = [0.1, 1, 10, 100]
results_c = []

for C in C_values:
    svr = SVR(kernel='rbf', C=C)
    svr.fit(X_train_scaled, y_train)
    y_pred = svr.predict(X_test_scaled)
    
    r2 = r2_score(y_test, y_pred)
    sv_count = len(svr.support_)
    
    results_c.append({'C': C, 'R2': r2, 'Support Vectors': sv_count})

results_c_df = pd.DataFrame(results_c)
print("=== C 파라미터 비교 ===")
print(results_c_df.to_string(index=False))

In [ ]:
# 검증
assert len(results_c_df) == 4, "4개 결과가 있어야 합니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. for 루프로 여러 C 값 테스트
2. 서포트 벡터 수도 함께 확인

**핵심 개념**:
- C 작음: 오차 허용적, 서포트 벡터 적음
- C 큼: 오차에 엄격, 서포트 벡터 많음 (과적합 위험)

**실무 팁**:
- C와 서포트 벡터 수의 관계 확인으로 과적합 판단

---

## Q4. gamma 파라미터 실험 ⭐⭐

**문제**: gamma=0.01, 0.1, 1 세 가지 값으로 SVR을 학습하고 성능을 비교하세요.

In [ ]:
# 정답 코드
gamma_values = [0.01, 0.1, 1]
results_gamma = []

for gamma in gamma_values:
    svr = SVR(kernel='rbf', gamma=gamma, C=1.0)
    svr.fit(X_train_scaled, y_train)
    y_pred = svr.predict(X_test_scaled)
    
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    results_gamma.append({'gamma': gamma, 'R2': r2, 'RMSE': rmse})

results_gamma_df = pd.DataFrame(results_gamma)
print("=== gamma 파라미터 비교 ===")
print(results_gamma_df.to_string(index=False))

In [ ]:
# 검증
assert len(results_gamma_df) == 3, "3개 결과가 있어야 합니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. gamma 값별 모델 학습
2. R2와 RMSE 모두 계산하여 비교

**핵심 개념**:
- gamma 작음: 넓은 영향 범위, 평활한 모델
- gamma 큼: 좁은 영향 범위, 복잡한 모델 (과적합 위험)

**실무 팁**:
- gamma='scale' (1 / (n_features * X.var())) 사용 권장

---

## Q5. 스케일링 효과 확인 ⭐⭐

**문제**: 스케일링 유무에 따른 SVR 성능 차이를 확인하세요.

In [ ]:
# 정답 코드
# 1. 스케일링 없이
svr_no_scale = SVR(kernel='rbf', C=1.0)
svr_no_scale.fit(X_train.values, y_train)
y_pred_no_scale = svr_no_scale.predict(X_test.values)
r2_no_scale = r2_score(y_test, y_pred_no_scale)

# 2. 스케일링 적용
svr_scaled = SVR(kernel='rbf', C=1.0)
svr_scaled.fit(X_train_scaled, y_train)
y_pred_scaled = svr_scaled.predict(X_test_scaled)
r2_scaled = r2_score(y_test, y_pred_scaled)

print("=== 스케일링 효과 ===")
print(f"스케일링 없음 - R2: {r2_no_scale:.4f}")
print(f"스케일링 적용 - R2: {r2_scaled:.4f}")
print(f"\n성능 향상: {r2_scaled - r2_no_scale:.4f}")

In [ ]:
# 검증
assert r2_scaled > r2_no_scale, "스케일링 후 성능이 더 좋아야 합니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 동일 모델로 스케일링 유무만 변경
2. R2 비교로 효과 확인

**핵심 개념**:
- SVR은 거리 기반 계산 (RBF 커널)
- 스케일 차이가 크면 일부 특성만 영향

**실무 팁**:
- SVR 사용 시 항상 StandardScaler 적용
- Pipeline으로 스케일링과 모델 결합 권장

---

## Q6. 선형 커널 SVR ⭐⭐⭐

**문제**: kernel='linear'인 SVR을 학습하고 RBF 커널과 성능을 비교하세요.

In [ ]:
# 정답 코드
# 1. 선형 커널 SVR
svr_linear = SVR(kernel='linear', C=1.0)
svr_linear.fit(X_train_scaled, y_train)
y_pred_linear = svr_linear.predict(X_test_scaled)
r2_linear = r2_score(y_test, y_pred_linear)

# 2. RBF 커널 SVR
svr_rbf = SVR(kernel='rbf', C=1.0)
svr_rbf.fit(X_train_scaled, y_train)
y_pred_rbf = svr_rbf.predict(X_test_scaled)
r2_rbf = r2_score(y_test, y_pred_rbf)

print("=== 커널 비교 ===")
print(f"Linear 커널 - R2: {r2_linear:.4f}")
print(f"RBF 커널    - R2: {r2_rbf:.4f}")

In [ ]:
# 검증
assert r2_linear > 0 and r2_rbf > 0
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 동일 C로 커널만 변경
2. R2 비교

**핵심 개념**:
- linear: 선형 관계만 학습, 빠름
- rbf: 비선형 패턴 학습, 느리지만 표현력 높음

**실무 팁**:
- 데이터 관계가 선형이면 linear가 더 적합
- 일반적으로 rbf로 시작

---

## Q7. 서포트 벡터 분석 ⭐⭐⭐

**문제**: SVR 모델의 서포트 벡터 수를 확인하고, C 값에 따라 어떻게 변하는지 분석하세요.

In [ ]:
# 정답 코드
C_values = [0.01, 0.1, 1, 10, 100]
sv_results = []

for C in C_values:
    svr = SVR(kernel='rbf', C=C)
    svr.fit(X_train_scaled, y_train)
    
    n_sv = len(svr.support_)
    ratio = n_sv / len(X_train) * 100
    
    sv_results.append({'C': C, 'Support Vectors': n_sv, 'Ratio (%)': ratio})

sv_df = pd.DataFrame(sv_results)
print("=== C에 따른 서포트 벡터 수 ===")
print(sv_df.to_string(index=False))

# 시각화
fig = px.bar(sv_df, x='C', y='Support Vectors', log_x=True,
             title='C에 따른 서포트 벡터 수 변화')
fig.show()

In [ ]:
# 검증
assert len(sv_df) == 5
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. svr.support_로 서포트 벡터 인덱스 접근
2. 전체 대비 비율 계산

**핵심 개념**:
- 서포트 벡터: 결정 경계에 영향을 주는 데이터 점
- C 큼 → 서포트 벡터 많음 → 복잡한 모델

**실무 팁**:
- 서포트 벡터가 전체의 50% 이상이면 과적합 의심

---

## Q8. GridSearchCV로 최적화 ⭐⭐⭐⭐

**문제**: C=[0.1, 1, 10], gamma=[0.01, 0.1], epsilon=[0.1, 0.5]를 GridSearchCV로 최적화하세요.

In [ ]:
# 정답 코드
# 1. 파라미터 그리드 정의
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.01, 0.1],
    'epsilon': [0.1, 0.5]
}

# 2. GridSearchCV 설정
svr_base = SVR(kernel='rbf')
grid_search = GridSearchCV(
    svr_base,
    param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

# 3. 학습
grid_search.fit(X_train_scaled, y_train)

# 4. 결과
print(f"\n최적 파라미터: {grid_search.best_params_}")
print(f"최적 CV RMSE: {np.sqrt(-grid_search.best_score_):.4f}")

# 5. 테스트 성능
best_svr = grid_search.best_estimator_
y_pred_best = best_svr.predict(X_test_scaled)
print(f"테스트 R2: {r2_score(y_test, y_pred_best):.4f}")

In [ ]:
# 검증
assert grid_search.best_params_['C'] in [0.1, 1, 10]
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. param_grid에 탐색할 파라미터 정의
2. GridSearchCV로 모든 조합 탐색
3. best_params_, best_score_로 결과 확인

**핵심 개념**:
- SVR의 세 파라미터(C, gamma, epsilon) 동시 최적화
- neg_mean_squared_error 사용 (sklearn 규칙)

**실무 팁**:
- 먼저 큰 범위로 탐색 → 좁은 범위로 정밀 탐색

---

## Q9. 교차 검증 ⭐⭐⭐⭐

**문제**: cross_val_score로 SVR의 5-fold 교차 검증을 수행하고 평균 R2를 계산하세요.

In [ ]:
# 정답 코드
# 1. SVR 모델
svr_cv = SVR(kernel='rbf', C=1.0, epsilon=0.1)

# 2. 5-fold 교차 검증
cv_scores = cross_val_score(
    svr_cv,
    X_train_scaled,
    y_train,
    cv=5,
    scoring='r2'
)

# 3. 결과
print("=== 5-Fold 교차 검증 ===")
print(f"각 Fold R2: {cv_scores.round(4)}")
print(f"평균 R2: {cv_scores.mean():.4f}")
print(f"표준편차: {cv_scores.std():.4f}")

In [ ]:
# 검증
assert len(cv_scores) == 5
assert cv_scores.mean() > 0.5
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. cross_val_score로 간편하게 CV 수행
2. 평균과 표준편차 계산

**핵심 개념**:
- 교차 검증: 모델 안정성 확인
- 표준편차 작음: 일관된 성능

**실무 팁**:
- CV 점수의 표준편차가 크면 데이터 분할에 민감 → 모델 개선 필요

---

## Q10. 모델 비교 종합 ⭐⭐⭐⭐⭐

**문제**: SVR(튜닝), Linear Regression, Random Forest 세 모델을 비교하고 Plotly로 시각화하세요.

In [ ]:
# 정답 코드
# 1. 모델 정의
models = {
    'Linear Regression': LinearRegression(),
    'SVR (튜닝)': best_svr,  # GridSearchCV 결과
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
}

# 2. 학습 및 평가
results = []

for name, model in models.items():
    if 'SVR' in name or 'Linear' in name:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    results.append({'Model': name, 'RMSE': rmse, 'R2': r2})

results_df = pd.DataFrame(results).sort_values('RMSE')
print("=== 모델 비교 ===")
print(results_df.to_string(index=False))

In [ ]:
# 3. Plotly 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['RMSE', 'R2 Score'])

fig.add_trace(
    go.Bar(x=results_df['Model'], y=results_df['RMSE'], marker_color='indianred',
           text=results_df['RMSE'].round(4), textposition='outside'),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=results_df['Model'], y=results_df['R2'], marker_color='seagreen',
           text=results_df['R2'].round(4), textposition='outside'),
    row=1, col=2
)

fig.update_layout(title='모델 성능 비교', height=400, showlegend=False)
fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
# 검증
assert len(results_df) == 3
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 여러 모델을 딕셔너리로 관리
2. 스케일링 필요 모델 분기 처리
3. make_subplots로 두 지표 동시 시각화

**핵심 개념**:
- 동일 데이터셋에서 공정한 비교
- SVR은 중소규모에서 경쟁력 있음

**실무 팁**:
- 데이터 크기, 해석 필요성 등을 고려하여 최종 모델 선택
- SVR은 소규모 고품질 데이터에서 강점

---

## 📊 학습 정리

### Part 1: SVR 기초 요약

| 개념 | 설명 | 핵심 포인트 |
|------|------|------------|
| SVR | SVM의 회귀 버전 | ε-tube 내 오차 무시 |
| epsilon (ε) | 오차 허용 범위 | 클수록 더 많은 오차 허용 |
| C | 페널티 강도 | 클수록 오차에 엄격 |
| 스케일링 | 필수 전처리 | StandardScaler 권장 |

### Part 2: SVR 활용 요약

| 개념 | 설명 | 권장 값 |
|------|------|--------|
| RBF 커널 | 비선형 패턴 학습 | 기본 선택 |
| gamma | 영향 범위 | 'scale' 또는 GridSearch |
| GridSearchCV | 최적 파라미터 탐색 | C, gamma, epsilon 동시 탐색 |

### 💡 실무 팁

1. **스케일링 필수**: SVR 전에 반드시 StandardScaler 적용
2. **데이터 크기 확인**: 대용량(>50,000)에서는 SVR 비추천
3. **GridSearchCV 활용**: C, gamma, epsilon 세 파라미터 동시 최적화
4. **서포트 벡터 확인**: 너무 많으면 과적합, 너무 적으면 과소적합
5. **커널 선택**: 비선형이면 RBF, 선형이면 linear (더 빠름)